In [172]:
import requests
import hashlib
import datetime
import string
import csv

In [38]:
str(datetime.datetime.now()).replace(" ", "").replace(".", "")

'2016-04-1622:14:37230653'

In [39]:
public_key = "bbca2c9c88fb26b6186f7adc5dafbc3e"
private_key = "f850ae27c606bc96740132921ab101c0e69d6511"


1460859277.6890051

In [42]:
#function setTSandHASH(timestamp, pub_key, priv_key)
m = hashlib.md5()
m.update("1abcd1234".encode('utf-8'))
m.hexdigest() == "ffd275c5130566a2916217b101f26150"

True

In [105]:
def get_TS_HASH( pub_key, priv_key):
    ts = str(datetime.datetime.now()).replace(" ", "").replace(".", "").replace("-", "").replace(":", "")
    m = hashlib.md5()
    h = ts + priv_key + pub_key
    m.update( h.encode('utf-8'))
    h = m.hexdigest()
    return ts, h
    

In [106]:
def make_url(base_url,limit,offset, pub_key, priv_key):
    ts, h = get_TS_HASH( pub_key, priv_key)
    get_url = base_url + '?ts=' + ts + '&limit=' + str(limit) + '&offset=' + str(offset) + "&apikey=" + pub_key + '&hash=' + h
    return get_url

In [107]:
events_url = "http://gateway.marvel.com:80/v1/public/events"

In [108]:
r = requests.get(make_url(events_url, 100, 0, public_key, private_key))

In [109]:
r.status_code

200

In [160]:
data = r.json()['data']

In [163]:
total =  int(data['total'])
total

69

In [152]:
data['results'][0]['title']

'Acts of Vengeance!'

In [153]:
data['results'][0]['id']

116

In [154]:
data['results'][0]['description']

'Loki sets about convincing the super-villains of Earth to attack heroes other than those they normally fight in an attempt to destroy the Avengers to absolve his guilt over inadvertently creating the team in the first place.'

In [155]:
data['results'][0]['next']['resourceURI'].split('/')[-1]

'240'

In [156]:
data['results'][0]['previous']['resourceURI'].split('/')[-1]

'233'

In [175]:
filepath = 'event.csv'
out_put = open(filepath, "w+", newline='')
datafile = csv.writer(out_put) 
header = ['id','title',  'description', 'next_event_id', 'prev_event_id']
datafile.writerow(header)

50

In [176]:
count = 0

In [177]:
r = requests.get(make_url(events_url, 100, count, public_key, private_key))
if(r.status_code != 200):
    print("something wrong!!! error code of %d"% r.status_code)
data = r.json()['data']
total = data['total']

In [178]:
while(count < total):
    count = count + data['count']
    for i in range(len(data['results'])):
        row = []
        event = data['results'][i]
        row.append(event['id'])
        row.append(event['title'])
        row.append(event['description'])
        try:
            row.append(event['next']['resourceURI'].split('/')[-1])
        except:
            row.append("null")
        try:
            row.append(event['previous']['resourceURI'].split('/')[-1])
        except:
            row.append("null")
            
        datafile.writerow(row)
        
    
    r = requests.get(make_url(events_url, 100, count, public_key, private_key))
        
    if(r.status_code != 200):
        print("something wrong!!! error code of %d"% r.status_code)
        break;
    data = r.json()['data']
        
        
out_put.close()       
            
        
        
    
    